The purpose of this notebook is to demonstrate how you can get relevant forecast-ready features for your locations at scale. To do this, we will use the [Features API](https://docs.predicthq.com/api/features). 

Make sure you have a predefined set of features and locations before running this notebook. The output is a ready-to-use dataframe of features for each of your locations.

# Steps

* [Setup](#setup)
* [1. Prepare Locations](#step-1-prepare-locations)
* [2. Features API: Get Features](#step-2-features-api-get-features)

# Setup

Complete the following steps before proceeding:

1. Install `requirements.txt`
2. Update `DATA_DIR` and `OUTPUT_DIR` as necessary
3. Replace `ACCESS_TOKEN` with a valid token (for help creating an access token, see [the API Quickstart](https://docs.predicthq.com/getting-started/api-quickstart))

In [1]:
# install requirements
# %pip install --user -r requirements.txt

In [2]:
import pandas as pd
import json
import os
from predicthq import Client

import features_api_utils as fau

In [3]:
DATA_DIR = "data"
OUTPUT_DIR = "output"

ACCESS_TOKEN = "REPLACE_WITH_ACCESS_TOKEN"

# REMOVE
import passwords

ACCESS_TOKEN = passwords.ACCESS_TOKEN

In [4]:
phq = Client(access_token=ACCESS_TOKEN)

# Step 1. Prepare Locations

**Features by Location**

If you have a list of features for each `location`, prepare a config file with the following information:

1. `features` or `important_features`: a list of features to get from the [Features API](https://docs.predicthq.com/api/features/get-features)

2. `place_id` or `lat`/`lon` for each location, with a unique location id as key

3. `industry`: (optional) the industry relevant to your locations (`accommodation`, `parking`, `food_and_beverage`, or `retail`), default is `other`

4. `min_phq_rank`: (optional) the PHQ Rank threshold for capturing relevant events around the location, default is set by industry

5. `start` and `end`: (optional) the date range to calculate features for, default is 2 years before and 3 months after today

**Features by Group of Locations**

If you have a list of features for multiple locations, prepare the above config file (excluding `features` or `important_features`). Then create a group config file with the following information:

1. `group_id`: the ID associated with a Beam Analysis Group

2. `locations`: list of location IDs associated with each contributing location within the group

3. `features` or `important_features`: a list of features to get from the [Features API](https://docs.predicthq.com/api/features/get-features)

See the example `config` and `group_config` below for how this should look. 

In [5]:
# load config file
with open(os.path.join(OUTPUT_DIR, "config_with_features.json"), "r") as f:
    config = json.load(f)

In [6]:
config

{'store_0': {'lat': 40.751582853596915,
  'lon': -73.98155897848956,
  'industry': 'restaurants',
  'analysis_name': 'store_0_analysis',
  'min_phq_rank': 30,
  'start': '2017-01-02',
  'end': '2019-12-31',
  'radius': 1.21,
  'radius_unit': 'mi',
  'analysis_id': 'ho8jzrnJLwU',
  'important_features': ['phq_attendance_performing_arts',
   'phq_rank_public_holidays',
   'phq_attendance_conferences',
   'phq_attendance_expos',
   'phq_attendance_community',
   'phq_attendance_school_holidays',
   'phq_attendance_festivals',
   'phq_attendance_concerts',
   'phq_attendance_sports',
   'phq_rank_observances',
   'phq_impact_severe_weather_air_quality_retail',
   'phq_impact_severe_weather_blizzard_retail',
   'phq_impact_severe_weather_cold_wave_retail',
   'phq_impact_severe_weather_cold_wave_snow_retail',
   'phq_impact_severe_weather_cold_wave_storm_retail',
   'phq_impact_severe_weather_dust_retail',
   'phq_impact_severe_weather_dust_storm_retail',
   'phq_impact_severe_weather_flood

In [7]:
# load group config file
with open(os.path.join(OUTPUT_DIR, "group_config_with_features.json"), "r") as f:
    group_config = json.load(f)

In [8]:
group_config

{'group_A': {'name': 'group_A_analysis',
  'locations': ['store_0', 'store_1'],
  'analysis_ids': ['ho8jzrnJLwU', 'KjQmR4C5wGo'],
  'group_id': 'BEnSsmK0CGg',
  'group_status': {'readiness_status': 'ready',
   'feature_importance_processing_completed': True},
  'important_features': ['phq_attendance_concerts',
   'phq_attendance_conferences',
   'phq_attendance_expos',
   'phq_attendance_festivals',
   'phq_rank_observances',
   'phq_attendance_performing_arts',
   'phq_rank_public_holidays',
   'phq_attendance_school_holidays',
   'phq_impact_severe_weather_air_quality_retail',
   'phq_impact_severe_weather_blizzard_retail',
   'phq_impact_severe_weather_cold_wave_retail',
   'phq_impact_severe_weather_cold_wave_snow_retail',
   'phq_impact_severe_weather_cold_wave_storm_retail',
   'phq_impact_severe_weather_dust_retail',
   'phq_impact_severe_weather_dust_storm_retail',
   'phq_impact_severe_weather_flood_retail',
   'phq_impact_severe_weather_heat_wave_retail',
   'phq_impact_sever

# Step 2. Features API: Get Features

Get your features of interest for all locations using [Features API](https://docs.predicthq.com/api/features). Features that are not important for a specific location appear as `NaN` and should be ignored.

**Features by Location**

In [9]:
# get features data
features_data = []
for location, info in config.items():
    print(f"Getting features data for {location}...")

    features_list = info.get("features", info.get("important_features", []))
    try:
        features = fau.get_features(info, features_list, phq)
        features.insert(0, "location", location)

        features_data.append(features)

        print(f"--- features retrieved")

    except Exception as e:
        print(f"--- an error occurred: {e}")
        continue

features_df = pd.concat(features_data)

# save features
features_df.to_csv(os.path.join(OUTPUT_DIR, "features.csv"), index=False)

Getting features data for store_0...
--- features retrieved
Getting features data for store_1...
--- features retrieved
Getting features data for store_2...
--- features retrieved


In [10]:
features_df

,location,date,phq_attendance_community,phq_attendance_concerts,phq_attendance_conferences,phq_attendance_expos,phq_attendance_festivals,phq_attendance_performing_arts,phq_attendance_school_holidays,phq_attendance_sports,...,phq_impact_severe_weather_dust_retail,phq_impact_severe_weather_dust_storm_retail,phq_impact_severe_weather_flood_retail,phq_impact_severe_weather_heat_wave_retail,phq_impact_severe_weather_hurricane_retail,phq_impact_severe_weather_thunderstorm_retail,phq_impact_severe_weather_tornado_retail,phq_impact_severe_weather_tropical_storm_retail,phq_rank_academic_exam,phq_rank_academic_holiday
0,store_0,2017-01-02,7104.0,32173.0,0.0,0.0,0.0,13564.0,0.0,19812.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,store_0,2017-01-03,7229.0,32403.0,0.0,0.0,0.0,27193.0,0.0,18006.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,store_0,2017-01-04,7481.0,780.0,0.0,0.0,0.0,38685.0,0.0,19812.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,store_0,2017-01-05,8004.0,3538.0,0.0,0.0,0.0,26021.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,store_0,2017-01-06,11340.0,6204.0,0.0,0.0,0.0,29884.0,0.0,19500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,store_2,2018-11-26,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
694,store_2,2018-11-27,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
695,store_2,2018-11-28,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
696,store_2,2018-11-29,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,...,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Features by Group of Locations**

In [13]:
# get features data
features_data = []
for group, group_info in group_config.items():
    print(f"Getting features data for {group}...")

    features_list = group_info.get("features", group_info.get("important_features", []))
    try:
        for location in group_info["locations"]:
            print(f"--- Getting features data for {location}...")

            info = config[location]

            try:
                features = fau.get_features(info, features_list, phq)
                features.insert(0, "group", group)
                features.insert(1, "location", location)
                features_data.append(features)

                print(f"    --- features retrieved")

            except Exception as e:
                print(f"--- an error occurred: {e}")
                continue

    except Exception as e:
        print(f"--- an error occurred: {e}")
        continue

features_df = pd.concat(features_data)

# save features
features_df.to_csv(os.path.join(OUTPUT_DIR, "group_features.csv"), index=False)

Getting features data for group_A...
--- Getting features data for store_0...
    --- features retrieved
--- Getting features data for store_1...
    --- features retrieved
Getting features data for group_B...
--- Getting features data for store_1...
    --- features retrieved
--- Getting features data for store_2...
    --- features retrieved


In [14]:
features_df

,group,location,date,phq_attendance_concerts,phq_attendance_conferences,phq_attendance_expos,phq_attendance_festivals,phq_attendance_performing_arts,phq_attendance_school_holidays,phq_attendance_sports,...,phq_impact_severe_weather_cold_wave_snow_retail,phq_impact_severe_weather_cold_wave_storm_retail,phq_impact_severe_weather_dust_retail,phq_impact_severe_weather_dust_storm_retail,phq_impact_severe_weather_flood_retail,phq_impact_severe_weather_heat_wave_retail,phq_impact_severe_weather_hurricane_retail,phq_impact_severe_weather_thunderstorm_retail,phq_impact_severe_weather_tornado_retail,phq_impact_severe_weather_tropical_storm_retail
0,group_A,store_0,2017-01-02,32173.0,0.0,0.0,0.0,13564.0,0.0,19812.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,group_A,store_0,2017-01-03,32403.0,0.0,0.0,0.0,27193.0,0.0,18006.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,group_A,store_0,2017-01-04,780.0,0.0,0.0,0.0,38685.0,0.0,19812.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,group_A,store_0,2017-01-05,3538.0,0.0,0.0,0.0,26021.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,group_A,store_0,2017-01-06,6204.0,0.0,0.0,0.0,29884.0,0.0,19500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,group_B,store_2,2018-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
694,group_B,store_2,2018-11-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
695,group_B,store_2,2018-11-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0
696,group_B,store_2,2018-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0
